In [ ]:
# import libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from PIL import Image
from io import BytesIO
from urllib import request


In [ ]:
# Initialize the TensorFlow Lite interpreter with the specified model
interpreter = tf.lite.Interpreter(model_path="model_2024_hairstyle.tflite")
# Allocate tensors for the interpreter
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [ ]:
# Get input and output details from the interpreter
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [4]:
# Print input and output details 
print("Input details:", input_details) 
print("Output details:", output_details)

Input details: [{'name': 'serving_default_input_layer:0', 'index': 0, 'shape': array([  1, 200, 200,   3], dtype=int32), 'shape_signature': array([ -1, 200, 200,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'StatefulPartitionedCall_1:0', 'index': 13, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [ ]:
# Get the input and output tensor indices from the interpreter details
input_index = input_details[0]['index']
output_index = output_details[0]['index']

In [ ]:
# Print the input and output tensor indices
print("Input index : ", input_index)
print("Output index : ", output_index)

Input index :  0
Output index :  13


In [ ]:
# Function to download an image from a URL
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()  # Read the response into a buffer
    stream = BytesIO(buffer)  # Create a BytesIO stream from the buffer
    img = Image.open(stream)  # Open the image from the stream
    return img

# Function to prepare an image by converting it to RGB and resizing it
def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')  # Convert image to RGB if not already in that mode
    img = img.resize(target_size, Image.NEAREST)  # Resize the image to the target size
    return img

In [ ]:
url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg" 

img = download_image(url)  # Download the image from the URL
target_size = (200, 200) 
resized_img = prepare_image(img, target_size) 
resized_img.show()

In [ ]:
# Function to convert an image to a numpy array
def image_to_array(img):
    return np.array(img)


In [ ]:
def preprocess_image(img):
    # Check if the image is in RGB mode, if not convert it to RGB
    if img.mode != 'RGB':
        img = img.convert('RGB')
    
    # Convert the image to a numpy array with float32 data type
    img_array = np.array(img, dtype=np.float32)
    
    # Normalize the pixel values to the range [0, 1]
    img_array = img_array / 255.0
    
    return img_array


In [ ]:
# preprocess the image
preprocessed_img = preprocess_image(resized_img)

In [ ]:
# extract the R channel of the first pixel
first_pixel_r = preprocessed_img[0, 0 ,0]
print("value of the first pixel, R channel ", first_pixel_r)

value of the first pixel, R channel  0.23921569


In [ ]:
# set the input tensor
interpreter.set_tensor(input_index, [preprocessed_img])

In [14]:
# Invoke the interpreter. 
interpreter.invoke() 
# Get the output. 
output = interpreter.get_tensor(output_index) 
print("Model output:", output)

Model output: [[0.8937725]]
